In [ ]:
from dcicutils import ff_utils
from functions.wfr import *
from functions.wfr_settings import *
from functions.notebook_functions import *

# tibanna = Tibanna(env=env)
my_env = 'data'
my_auth = get_key('koray_data')


In [ ]:
# url for repliseq exps
exp_types = ['Repli-seq']
set_url = '/search/?'+'&'.join(['experiments_in_set.experiment_type='+i for i in exp_types])+ \
          '&type=ExperimentSetReplicate&limit=all'
run_sets = ff_utils.search_metadata(set_url ,key=my_auth)

add_pc = False
add_rel = False
add_wfr = True

counter = 0
all_sets = len(run_sets)
print(str(all_sets)+' total number of sets')

run_sets = [i for i in run_sets if "RepliSeq_Pipeline_0.2.5"  not in i.get('completed_processes', [])]

print(str(all_sets-len(run_sets))+ ' sets completed')


for a_set in run_sets: 
    counter += 1
    print()
    fastqpairs, organism, enzyme, bwa_ref, chrsize_ref, enz_ref, f_size, lab = find_pairs(a_set, my_env, lookfor = 'single')
    if not bwa_ref or not chrsize_ref:
        print(counter, a_set['accession'], organism, enzyme, 'skipping set with not chrsize/bwa index')
        continue  
    print(counter, a_set['accession'], enzyme, organism)      
    # cycle through the experiments
    for exp in fastqpairs.keys():
        if not fastqpairs.get(exp):
            print(exp, 'does not have any fastq files')
            continue
        # Check Part 1 and See if all are okay
        exp_bams = []
        part1 = 'done'
        # part2 = 'done'
        for pair in fastqpairs[exp]:
            #############
            pair_resp = ff_utils.get_metadata(pair, key=my_auth)
            attributions = get_attribution(pair_resp)
            report = get_wfr_out(pair, 'repliseq-parta 0.2.5', my_auth)
            # if run is not successful
            if report['status'].startswith('no'):
                part1 = 'not done'
                if add_wfr:
                    inp_f = {'fastq':pair, 'chromsizes':chrsize_ref, 'bwaIndex':bwa_ref}
                    name_tag = pair.split('/')[2]
                    run_missing_wfr(step_settings('repliseq-parta', organism, attributions), inp_f, name_tag,my_auth, my_env)
            elif report['status'] == 'running':
                part1 = 'still running'
                print('part1 still running')
            # if successful
            else:
                assert report['status'] == 'complete'
                if add_pc:
                    # TODO check if the files already in processed files field.
                    # don't do it if they are already carried there.
                    add_preliminary_processed_files(exp,
                                                    [report['bam'], report['bg']],
                                                    my_auth,
                                                    run_type='repliseq')
        # stop progress to part2 
        if part1 is not 'done':
            print(exp, 'has missing Part1 runs')
            part2 = 'not ready'
            part3 = 'not ready'
            continue
        print(exp, 'part1 complete')
        
        

        
